In [1]:
import pandas as pd
import numpy as np
import json

# image and station data cleaning

In [3]:
stations_by_name=pd.read_json("raw_data/stations_by_name_with_alias.json",orient='index')
stations_by_name["img"]=stations_by_name.ST_NAME_EN+".png"
stations_by_name=stations_by_name[stations_by_name.index!="严御路"]
stations_by_name=stations_by_name[stations_by_name.index!="外高桥保税区北"]
stations_by_name=stations_by_name[stations_by_name.index!="李子园路"]
stations_by_name=stations_by_name[stations_by_name.index!="淞浜路"]
stations_by_name=stations_by_name[stations_by_name.index!="外高桥保税区南"]
stations_by_name=stations_by_name[stations_by_name.index!="上海大学站"]
stations_by_name=stations_by_name[stations_by_name.index!="上海野生动物园"]
stations_by_name=stations_by_name.drop_duplicates(subset=None, keep='first', inplace=False)
stations_by_name


,LAT,LINE_NO,LNG,SERIAL_NO,ST_HORIZONTAL,ST_NAME,ST_NAME_EN,ST_NO,TRANSFER_TAG,X,Y,img
马陆,31.319586,11,121.276874,9,高架,马陆,Malu,1135,普通站,200,226,Malu.png
共康路,31.319002,1,121.447024,7,高架,共康路,GongkangRoad,132,普通站,519,287,GongkangRoad.png
淞虹路,31.218207,2,121.359583,4,地下,淞虹路,SonghongRoad,237,普通站,214,476,SonghongRoad.png
刘行,31.357454,7,121.362490,4,地下,刘行,Liuhang,724,普通站,394,146,Liuhang.png
江浦路,31.274889,8,121.518402,24,地下,江浦路,JiangpuRoad,843,普通站,804,389,JiangpuRoad.png
海天三路,31.168483,2,121.796847,29,地面,海天三路,HaitiansanRoad,262,普通站,917,846,HaitiansanRoad.png
华夏中路,31.175725,16,121.583032,2,高架,华夏中路,MiddleHuaxiaRoad,1622,普通站,,,MiddleHuaxiaRoad.png
中潭路,31.254567,4,121.441025,9,高架,中潭路,ZhongtanRoad,409,换乘站,470,433,ZhongtanRoad.png
嘉定新城,31.329997,11,121.254258,8,高架,嘉定新城,JiadingXincheng,1134,普通站,212,208,JiadingXincheng.png
丰庄,31.242491,13,121.355152,3,地下,丰庄,Fengzhuang,1323,普通站,272,380,Fengzhuang.png


In [4]:
stations_by_name.to_json(r"/Users/HYT/Desktop/CS573 Data Visualization/git/gradfinal/data/stations_by_name.json",orient='index',force_ascii=False)


# Moving Line Data Generation

In [101]:
connections=pd.read_json("connections_by_station_name.json",orient='column')

In [102]:
def stepStation(ForthTime,station):  
    '''
    Input:
        ForthTime: A list of transition time between each station. 
                   Length: n
                  e.g.:[2,2,3,3,3,3,2,2,2,2,2,3,2,2,3,2,3,2,2,3,3,2,2,3,3,2,2]
        station: A station list
                 Length: n+1
    Output: The record of the position of this metro every min when the metro is running on the line.
            e.g. : [[['A', 'A'], 0],
                    [['A', 'B'], 0.3333333333333333],
                    [['A', 'B'], 0.6666666666666666],
                    [['B', 'B'], 0],
                    [['B', 'C'], 0.5],
                    ...]
                    Here A,B,C,... denote station names.
    '''
    StepStation = []
    currentStationIndex = 0
    iterTime=iter(ForthTime)
    steptime = next(iterTime)
    currentratio = 0
    stepratio = 1 / steptime
    TotalTime = sum(ForthTime)
    for i in range(TotalTime+1):
        if currentratio==0:
            StepStation.append([[station[currentStationIndex],station[currentStationIndex]],currentratio])
        else:
            StepStation.append([[station[currentStationIndex],station[currentStationIndex+1]],currentratio])
        steptime-=1
        currentratio+=stepratio
        if steptime==0:
            try:
                steptime = next(iterTime)
                stepratio =1 / steptime
                currentStationIndex+=1
                currentratio=0
            except:
                currentStationIndex+=1
                StepStation.append([[station[currentStationIndex],station[currentStationIndex]],0]) 
                break
    return StepStation
    

In [103]:
def MtoRDepartureTime(sectionNo,OpenTime,EarlyPeak,MediumPeak,EveningPeak):
    DepartureTime = []
    #Before Early peak
    t=OpenTime[0]
    while (t<EarlyPeak[0]):
        DepartureTime.append(t)
        t=t+QueueTime(0,sectionNo)
    #Early peak
    while (t<MediumPeak[0]):
        DepartureTime.append(t)
        t=t+QueueTime(1,sectionNo)
    #MediumPeak
    while (t<EveningPeak[0]):
        DepartureTime.append(t)
        t=t+QueueTime(2,sectionNo)
    #EveningPeak
    while (t<EveningPeak[1]):
        DepartureTime.append(t)
        t=t+QueueTime(3,sectionNo)
    #19:00 - 23:37 with DurationD
    while (t<OpenTime[1]):
        DepartureTime.append(t)
        t=t+QueueTime(0,sectionNo)
    return DepartureTime

In [104]:

def SectionTimeAndPosition(TimeAndPosition,DepartureTime,StepPosition,lineNo,sectionNo,direction):
    for T in DepartureTime:
            iterStepPosition = iter(StepPosition)
            i=T
            flag = 1
            while flag:
                try:
                    try:
                        currentP = next(iterStepPosition)
                    except:
                        flag = 0
                    TimeAndPosition[i].append([i,T,currentP,lineNo,sectionNo,direction])
                except:
                    TimeAndPosition[i] = []
                    TimeAndPosition[i].append([i,T,currentP,lineNo,sectionNo,direction])
                i+=1
                
def TimeAndPosition(lineNo,senctionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak):
    TimeAndPosition = dict()
    for sectionNo in range(senctionLength):
        ForceDepartureTime = MtoRDepartureTime(sectionNo,ForthOpenTime,EarlyPeak,MediumPeak,EveningPeak)
        BackDepartureTime = MtoRDepartureTime(sectionNo,ForthOpenTime,EarlyPeak,MediumPeak,EveningPeak)
        ForceStepPosition = stepStation(ForthTime[sectionNo],station[sectionNo])
        BackStepPosition = ForceStepPosition[::-1]
        SectionTimeAndPosition(TimeAndPosition,ForceDepartureTime,ForceStepPosition,lineNo,sectionNo,'f')
        SectionTimeAndPosition(TimeAndPosition,BackDepartureTime,BackStepPosition,lineNo,sectionNo,'b')
    return TimeAndPosition

## Line One
Forth: 富锦路 - 上海火车站 - 莘庄 (FuJin Road - Shanghai train station - ZiZhuang)

Back: 莘庄 - 上海火车站 - 富锦路 (ZiZhuang - Shanghai train station - FuJin Road)

### Spatial Record 

In [106]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==1][0]).tolist()
stationTotal.append("莘庄")
#富锦路 - 上海火车站, 上海火车站 - 莘庄
station = [stationTotal[0:stationTotal.index("上海火车站")+1],stationTotal[stationTotal.index("上海火车站"):]]
sectionLength = 2

# Commuting time between each station on each section.
BackTimeTotal = [2,2,3,3,3,3,2,2,2,2,2,3,2,2,3,2,3,2,2,3,3,2,2,3,3,2,2]
ForthTimeTotal=BackTimeTotal[::-1]
ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),stepStation(ForthTime[1],station[1])]

### Time Record -  Monday to Thursday

In [107]:
ForthOpenTime = [330,1414]   #05:30 - 23:34
BackOpenTime = [330,1417]    #05:30 - 23:37

EarlyPeak = [420,539]   #7:00 - 8:59
MediumPeak =[540,1019] #9:00 - 16:59
EveningPeak = [1020,1140] #17:00 - 18:59

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        return np.random.randint(2,4)
    elif p==2:
        if sectionNo==0:
            return 6   #富锦路 - 上海火车站
        else:
            return 4   #上海火车站 - 莘庄
    elif p==3:
        return 3
    else:
        return np.random.randint(4,10)

LineOne = TimeAndPosition(1,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineOne.json', 'w') as fp:
    json.dump(LineOne, fp,ensure_ascii=False)

## Line Two
Forth: 徐泾东 - 淞虹路 - 广兰路 - 浦东国际机场

Back: 浦东国际机场 - 广兰路 - 淞虹路 - 徐泾东

### Spatial Record 

In [108]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==2][0]).tolist()
stationTotal.append("浦东国际机场")
#徐泾东 - 淞虹路 - 广兰路 - 浦东国际机场
station = [stationTotal[0:stationTotal.index("淞虹路")+1],
           stationTotal[stationTotal.index("淞虹路"):stationTotal.index("广兰路")+1],
           stationTotal[stationTotal.index("广兰路"):]]
sectionLength = 3

# Commuting time between each station on each section.
ForthTimeTotal = [3,2,7,2,2,3,3,2,2,3,3,2,3,2,3,2,3,2,4,2,4,4,3,3,4,3,5,6,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):(len(station[0])+len(station[1])-2)],
           ForthTimeTotal[(len(station[0])+len(station[1])-2):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1]),
               stepStation(ForthTime[2],station[2])]

### Time Record -  Monday to Thursday

In [109]:
ForthOpenTime = [328, 1381]   #05:28 - 23:01
BackOpenTime = [360, 1442]  #06:00 - 00:02

EarlyPeak = [450,570 ]  #7:30 - 9:30
MediumPeak = [570,1020 ]  #9:30 - 17:00
EveningPeak = [1020, 1230 ]   #17:00 - 20:30

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==1:
            return 3
        else:
            return np.random.randint(4,6)           
    elif p==2:
        if sectionNo==1:
            return 4
        else:
            return 8
    elif p==3:
        if sectionNo==1:
            return np.random.randint(3,5)
        else:
            return 7
    else:
        return np.random.randint(5,13)

LineTwo = TimeAndPosition(2,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineTwo.json', 'w') as fp:
    json.dump(LineTwo, fp,ensure_ascii=False)

## Line Three
Forth: 上海南站 - 长江南路 - 江杨北路

Back: 江杨北路 - 长江南路 - 上海南站

### Spatial Record 

In [110]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==3][0]).tolist()
stationTotal.append("上海南站")
stationTotal=stationTotal[::-1]
#上海南站 - 长江南路 - 江杨北路
station = [stationTotal[0:stationTotal.index("长江南路")+1],
           stationTotal[stationTotal.index("长江南路"):]]
sectionLength = 2

# Commuting time between each station on each section.
ForthTimeTotal = [2,3,2,2,2,3,2,2,2,3,2,3,3,2,3,2,2,3,2,3,2,3,2,2,3,2,2,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1])]

### Time Record -  Monday to Thursday

In [111]:
ForthOpenTime = [325, 1417]   #05:25 - 23:37
BackOpenTime = [325, 1422]  #05:25 - 23:42

EarlyPeak = [450,570 ]  #7:30 - 9:30
MediumPeak = [570,1020 ]  #9:30 - 17:00
EveningPeak = [1020, 1170 ]   #17:00 - 19:30

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==0:
            return 4
        else:
            return int(np.random.choice([6,7], p=[1/3,2/3]) )
    elif p==2:
        if sectionNo==0:
            return 7   
        else:
            return 14   
    elif p==3:
        if sectionNo==0:
            return 5   
        else:
            return 10 
    else:
        if sectionNo==0:
            return np.random.randint(7,11)
        else:
            return np.random.randint(14,18)

LineThree = TimeAndPosition(3,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineThree.json', 'w') as fp:
    json.dump(LineThree, fp,ensure_ascii=False)

## Line Five
Forth: 闵行开发区 - 东川路 - 莘庄

Back: 莘庄 - 东川路 - 闵行开发区

### Spatial Record 

In [112]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==5][0]).tolist()
stationTotal.append("闵行开发区")
stationTotal=stationTotal[::-1]
#闵行开发区 - 东川路 - 莘庄
station = [stationTotal[0:stationTotal.index("东川路")+1],
           stationTotal[stationTotal.index("东川路"):]]
sectionLength = 2

# Commuting time between each station on each section.
ForthTimeTotal = [2,2,3,2,2,3,4,3,2,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1])]

### Time Record -  Monday to Thursday

In [113]:
ForthOpenTime = [350, 1350]   #05:50 - 22:30
BackOpenTime = [350, 1381]  #05:50 - 23:01

EarlyPeak = [420,570 ]  #7:00 - 9:30
MediumPeak = [570,1020 ]  #9:30 - 17:00
EveningPeak = [1020, 1200 ]   #17:00 - 20:00

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==0:
        return np.random.randint(7,11)
    if sectionNo==0:
            return np.random.randint(7,9)
    else:
        if p==2:
            return 6
        else:
            return 4

LineFive = TimeAndPosition(5,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineFive.json', 'w') as fp:
    json.dump(LineFive, fp,ensure_ascii=False)

## Line Six
Forth: 东方体育中心 - 高青路 - 巨峰路 - 港城路

Back: 港城路 - 巨峰路 - 高青路 - 东方体育中心

### Spatial Record 

In [114]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==6][0]).tolist()
stationTotal.append("港城路")
#东方体育中心 - 高青路 - 巨峰路 - 港城路
station = [stationTotal[0:stationTotal.index("高青路")+1],
           stationTotal[stationTotal.index("高青路"):stationTotal.index("巨峰路")+1],
           stationTotal[stationTotal.index("巨峰路"):]]
sectionLength = 3

# Commuting time between each station on each section.
ForthTimeTotal = [2,2,2,3,2,3,2,3,3,2,3,2,2,2,3,2,3,2,2,2,3,2,2,3,3,2,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):(len(station[0])+len(station[1])-2)],
           ForthTimeTotal[(len(station[0])+len(station[1])-2):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1]),
               stepStation(ForthTime[2],station[2])]

### Time Record -  Monday to Thursday

In [115]:
ForthOpenTime = [330, 1415]   #05:30 - 23:35
BackOpenTime = [330, 1415]  #05:30 - 23:34

EarlyPeak = [460, 530]  #7:40 - 8:50
MediumPeak = [530, 990]  #8:50 - 16:30
EveningPeak = [990, 1140]   #16:30 - 19:00

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==0:
            return int(np.random.choice([4,5], p=[2/3,1/3]) )
        elif sectionNo==1:
            return 2
        else:
            return int(np.random.choice([4,5], p=[5/6,1/6]) )          
    elif p==2:
        if sectionNo==1:
            return np.random.randint(4,6)
        else:
            return 9
    elif p==3:
        if sectionNo==1:
            return np.random.randint(2,4)
        else:
            return np.random.randint(7,9)
    else:
        if sectionNo==1:
            return np.random.randint(5,7)
        else:
            return np.random.randint(10,12)

LineSix = TimeAndPosition(6,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineSix.json', 'w') as fp:
    json.dump(LineSix, fp,ensure_ascii=False)

## Line Seven
Forth: 美兰湖 - 上大路 - 杨高南路 - 花木路

Back: 花木路 - 杨高南路 - 上大路 - 美兰湖

### Spatial Record 

In [116]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==7][0]).tolist()
stationTotal.append("花木路")
#美兰湖 - 上大路 - 杨高南路 - 花木路
station = [stationTotal[0:stationTotal.index("上大路")+1],
           stationTotal[stationTotal.index("上大路"):stationTotal.index("杨高南路")+1],
           stationTotal[stationTotal.index("杨高南路"):]]
sectionLength = 3

# Commuting time between each station on each section.
ForthTimeTotal = [2,4,2,3,4,2,2,2,3,2,2,2,2,2,3,3,2,2,2,3,2,2,4,2,2,2,2,3,2,2,4,2]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):(len(station[0])+len(station[1])-2)],
           ForthTimeTotal[(len(station[0])+len(station[1])-2):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1]),
               stepStation(ForthTime[2],station[2])]

### Time Record -  Monday to Thursday

In [117]:
ForthOpenTime = [330, 1413]   #05:30 - 23:33
BackOpenTime = [330, 1428]  #05:30 - 23:48

EarlyPeak = [460,540 ]  #7:40 - 9:00
MediumPeak = [540,1080 ]  #9:00 - 18:00
EveningPeak = [1080, 1170 ]   #18:00 - 19:30

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==1:
            return int(np.random.choice([2,3], p=[1/2,1/2]) )
        else:
            return int(np.random.choice([5,6], p=[2/3,1/3]) )          
    elif p==2:
        if sectionNo==0:
            return 11
        else:
            return int(np.random.choice([5,6], p=[1/2,1/2]) )
    elif p==3:
        if sectionNo==0:
            return int(np.random.choice([6,7], p=[1/6,5/6]) )
        else:
            return int(np.random.choice([4,5], p=[1/3,2/3]) )
    else:
        if sectionNo==0:
            return np.random.randint(10,12)
        else:
            return np.random.randint(5,9)

LineSeven = TimeAndPosition(7,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineSeven.json', 'w') as fp:
    json.dump(LineSeven, fp,ensure_ascii=False)

## Line Eight
Forth: 沈杜公路 - 东方体育中心 - 延吉中路 - 市光路

Back: 市光路 - 延吉中路 - 东方体育中心 - 沈杜公路

### Spatial Record 

In [118]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==8][0]).tolist()
stationTotal.append("市光路")
#沈杜公路 - 东方体育中心 - 延吉中路 - 市光路
station = [stationTotal[0:stationTotal.index("东方体育中心")+1],
           stationTotal[stationTotal.index("东方体育中心"):stationTotal.index("延吉中路")+1],
           stationTotal[stationTotal.index("延吉中路"):]]
sectionLength = 3

# Commuting time between each station on each section.
ForthTimeTotal = [2,2,2,4,3,4,2,2,2,2,3,2,2,2,2,2,2,3,2,3,2,2,2,2,2,2,2,2,2]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):(len(station[0])+len(station[1])-2)],
           ForthTimeTotal[(len(station[0])+len(station[1])-2):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1]),
               stepStation(ForthTime[2],station[2])]

### Time Record -  Monday to Thursday

In [119]:
ForthOpenTime = [330, 1416]   #05:30 - 23:36
BackOpenTime = [330, 1350]  #05:30 - 22:30

EarlyPeak = [435,525 ]  #7:15 - 8:45
MediumPeak = [525,1020 ]  #8:45 - 17:00
EveningPeak = [1020, 1160 ]   #17:00 - 19:20

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==2:
            return int(np.random.choice([3,4], p=[2/3,1/3]) )
        else:
            return int(np.random.choice([2,3], p=[2/3,1/3]) )          
    elif p==2:
        if sectionNo==1:
            return 4
        else:
            return 8
    elif p==3:
        if sectionNo==2:
            return int(np.random.choice([5,6], p=[1/2,1/2]) )
        else:
            return int(np.random.choice([2,3], p=[1/4,3/4]) )
    else:
        if sectionNo==1:
            return np.random.randint(5,7)
        else:
            return np.random.randint(10,13)

LineEight = TimeAndPosition(8,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineEight.json', 'w') as fp:
    json.dump(LineEight, fp,ensure_ascii=False)

## Line Nine
Forth: 松江南站 - 佘山 - 杨高中路

Back: 杨高中路 - 佘山 - 松江南站

### Spatial Record 

In [120]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==9][0]).tolist()
stationTotal.append("杨高中路")
#松江南站 - 佘山 - 杨高中路
station = [stationTotal[0:stationTotal.index("佘山")+1],
           stationTotal[stationTotal.index("佘山"):]]
sectionLength = 2

# Commuting time between each station on each section.
ForthTimeTotal = [4,3,3,3,5,3,4,7,4,2,3,3,3,7,3,4,2,2,2,3,2,3,3,3,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1])]

### Time Record -  Monday to Thursday

In [121]:
ForthOpenTime = [340, 1452]   #05:40 - 00:12
BackOpenTime = [330, 1454]  #05:30 - 00:14

EarlyPeak = [480,540 ]  #8:00 - 9:00
MediumPeak = [540,1050 ]  #9:00 - 17:30
EveningPeak = [1050, 1140 ]   #17:30 - 19:00

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==0:
            return 3
        else:
            return int(np.random.choice([1,2], p=[1/12,11/12]) )
    elif p==2:
        return int(np.random.choice([5,6], p=[1/2,1/2]) )  
    elif p==3:
        if sectionNo==0:
            return 6  
        else:
            return 3
    else:
        return np.random.randint(8,11)

LineNine = TimeAndPosition(9,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineNine.json', 'w') as fp:
    json.dump(LineNine, fp,ensure_ascii=False)

## Line Twelve
Forth: 曲阜路 - 巨峰路 - 金海路

Back: 金海路 - 巨峰路 - 曲阜路

### Spatial Record 

In [122]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==12][0]).tolist()
stationTotal.append("金海路")
#曲阜路 - 巨峰路 - 金海路
station = [stationTotal[0:stationTotal.index("巨峰路")+1],
           stationTotal[stationTotal.index("巨峰路"):]]
sectionLength = 2

# Commuting time between each station on each section.
ForthTimeTotal = [3,3,2,2,2,2,3,2,2,3,3,2,2,3,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1])]

### Time Record -  Monday to Thursday

In [123]:
ForthOpenTime = [357, 1427]   #05:57 - 23:47
BackOpenTime = [330, 1387]  #05:30 - 23:07

EarlyPeak = [480,540 ]  #8:00 - 9:00
MediumPeak = [540,990 ]  #9:00 - 16:30
EveningPeak = [990, 1140 ]   #16:30 - 19:00

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==0:
            return 3
        else:
            return 6
    elif p==2:
        return 6  
    elif p==3:
        if sectionNo==0:
            return int(np.random.choice([4,5], p=[5/6,1/6]) ) 
        else:
            return int(np.random.choice([8,9], p=[2/3,1/3]) ) 
    else:
        return np.random.randint(8,11)

LineTwelve = TimeAndPosition(12,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineTwelve.json', 'w') as fp:
    json.dump(LineTwelve, fp,ensure_ascii=False)

## Line Thirteen
Forth: 金运路 - 长寿路

Back: 长寿路 - 金运路

### Spatial Record 

In [124]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==13][0]).tolist()
stationTotal.append("长寿路")
#曲阜路 - 巨峰路 - 金海路
station = [stationTotal]
sectionLength = 1

# Commuting time between each station on each section.
ForthTimeTotal = [2,4,3,2,3,4,2,3,2]

ForthTime=[ForthTimeTotal]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0])]

### Time Record -  Monday to Thursday

In [125]:
ForthOpenTime = [330, 1400]   #05:30 - 23:20
BackOpenTime = [372, 1421]  #06:12 - 23:41

EarlyPeak = [450,570 ]  #7:30 - 9:30
MediumPeak = [570,1020 ]  #9:30 - 17:00
EveningPeak = [1020, 1170 ]   #17:00 - 19:30

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        return 3
    elif p==2:
        return 6  
    elif p==3:
        return 3
    else:
        return np.random.randint(6,11)

LineThirteen = TimeAndPosition(13,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineThirteen.json', 'w') as fp:
    json.dump(LineThirteen, fp,ensure_ascii=False)

## Line Sixteen
Forth: 龙阳路 - 惠南东 - 滴水湖

Back: 滴水湖 - 惠南东 - 龙阳路

### Spatial Record 

In [126]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==16][0]).tolist()
stationTotal.append("滴水湖")
#龙阳路 - 惠南东 - 滴水湖
station = [stationTotal[0:stationTotal.index("惠南东")+1],
           stationTotal[stationTotal.index("惠南东"):]]
sectionLength = 2

# Commuting time between each station on each section.
ForthTimeTotal = [4,4,4,4,3,4,5,6,5,9,6,3]

ForthTime=[ForthTimeTotal[:(len(station[0])-1)],
           ForthTimeTotal[(len(station[0])-1):]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1])]

### Time Record -  Monday to Thursday

In [127]:
ForthOpenTime = [350, 1422]   #05:50 - 23:42
BackOpenTime = [360, 1421]  #06:00 - 23:41

EarlyPeak = [430,540 ]  #7:10 - 9:00
MediumPeak = [540,1020 ]  #9:00 - 17:00
EveningPeak = [1020, 1260 ]   #17:00 - 21:00

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        if sectionNo==0:
            return 4
        else:
            return np.random.randint(4,6)
    elif p==2:
        return 8  
    elif p==3:
        return 4
    else:
        return np.random.randint(8,13)

LineSixteen = TimeAndPosition(16,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineSixteen.json', 'w') as fp:
    json.dump(LineSixteen, fp,ensure_ascii=False)

## Line Four
Forth: 上海体育场 — 东安路 - 宜山路 - 上海体育场

Back: 上海体育场 — 宜山路 - 东安路 - 上海体育场

### Spatial Record 

In [28]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
stationTotal = (connections[connections[2]==4][0]).tolist()
stationTotal.append("上海体育馆")
#上海体育场 — 东安路 - 宜山路 - 上海体育场
station = [stationTotal]
sectionLength = 1

# Commuting time between each station on each section.
ForthTimeTotal = [2,3,2,3,2,3,2,2,2,3,2,2,2,2,3,3,3,2,3,1,3,2,2,4,2]

ForthTime=[ForthTimeTotal]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0])]

### Time Record -  Monday to Thursday

In [29]:
ForthOpenTime = [334, 1352]   #05:34 - 22:32
BackOpenTime = [330, 1405]  #05:30 - 23:25

EarlyPeak = [450,540 ]  #7:30 - 9:00
MediumPeak = [540,1020 ]  #9:00 - 17:00
EveningPeak = [1020, 1170 ]   #17:00 - 19:30

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if p==1:
        return 4
    elif p==2:
        return 7 
    elif p==3:
        return 5
    else:
        return np.random.randint(7,16)

LineFour = TimeAndPosition(4,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineFour.json', 'w') as fp:
    json.dump(LineFour, fp,ensure_ascii=False)

## Line Ten
Forth: 嘉定北 - 嘉定新城 , 花桥 - 嘉定新城 , 嘉定新城 - 迪士尼

Back: 

### Spatial Record 

In [84]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
a=connections[connections[2]==10][:3][0].tolist()
a.append("龙溪路")
b=connections[connections[2]==10][3:8][0].tolist()
c=connections[connections[2]==10][6:][1][::-1].tolist()
#航中路 — 龙溪路
station = [a,b,c]
sectionLength = 3

# Commuting time between each station on each section.
ForthTime=[[2,2,3],
           [1,3,3,2],
           [1,2,2,2,2,2,2,2,2,2,2,3,2,2,2,3,2,2,3,2,1,3,2]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1]),
               stepStation(ForthTime[2],station[2])]

### Time Record -  Monday to Thursday

In [70]:
ForthOpenTime = [330, 1359]   #05:30 - 22:39
BackOpenTime = [378, 1400]  #06:18 - 23:20

EarlyPeak = [420,540]  #7:00 - 9:00
MediumPeak = [540,540]
EveningPeak = [540,1350 ]   #9:00 - 22:30

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if sectionNo==0:
            return 10
    if sectionNo==1:
        if p==1:
            return int(np.random.choice([6,7], p=[1/3,2/3]) )
        elif p==2:
            return int(np.random.choice([4,5], p=[43/60,17/60]) )
        elif p==3:
            return 6
        else:
            return 10
    else:
        return 5
    
LineTen = TimeAndPosition(10,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineTen.json', 'w') as fp:
    json.dump(LineTen, fp,ensure_ascii=False)

## Line Eleven
Forth: 嘉定北-嘉定新城, 花桥-嘉定新城, 嘉定新城-罗山路

Back: 

### Spatial Record 

In [94]:
'''
    This block output a list: StepStation.
    Assume that for each section in this line, there is a metro running in forth direction.
    StepStation: the position of this metro at every minute for each section.
        e.g. StepStation = [section0 , section1].
             section0 = [[['A', 'A'], 0],
                            [['A', 'B'], 0.3333333333333333],
                            [['A', 'B'], 0.6666666666666666],
                            [['B', 'B'], 0],
                            [['B', 'C'], 0.5],
                            ...]
                            Here A,B,C,... denote station names.
'''
# Station List
a=connections[connections[2]==11][7:11][0].tolist()
b=connections[connections[2]==11][0:7][0].tolist()
b.append("嘉定新城")
c=connections[connections[2]==11][9:][1].tolist()
#station
station = [a,b,c]
sectionLength = 3

# Commuting time between each station on each section.
ForthTime=[[2,4,3],
           [2,3,2,3,3,6,4],
           [3,4,4,2,2,3,2,3,2,2,2,3,3,3,3,3,2,2,3,5,2,2,6,3]]

#For each section on one line, report the record of the position of one metro every min.
StepStation = [stepStation(ForthTime[0],station[0]),
               stepStation(ForthTime[1],station[1]),
               stepStation(ForthTime[2],station[2])]

### Time Record -  Monday to Thursday

In [95]:
ForthOpenTime = [338, 1330]   #05:38 - 22:10
BackOpenTime = [393, 1439]  #06:33 - 23:59

EarlyPeak = [450,540 ]  #7:30 - 9:00
MediumPeak  = [540, 1020 ]   #9:00 - 17:00
EveningPeak = [1020, 1140 ]   #17:00 - 19:00

def QueueTime(p,sectionNo):
    '''
    Input: 
        (p = 1) : Early Peak
        (p = 2) : Medium Peak
        (p = 3) : Evening Peak
        (P = 0) : Other Time
        
        sectionNo: 0,1,..
    Output:
        QueueTime for departure metro during for a given time interval on a given section.
    '''
    if sectionNo==2:
        if p==1:
            return 3
        elif p==2:
            return 6
        elif p==3:
            return 7
        else: return 6
    else:
        if p==1:
            return 6
        elif p==2:
            return 12
        elif p==3:
            return 7
        else:
            return np.random.randint(10,13)
    
LineEleven = TimeAndPosition(11,sectionLength,ForthOpenTime,BackOpenTime,EarlyPeak,MediumPeak,EveningPeak)
with open('position/LineEleven.json', 'w') as fp:
    json.dump(LineEleven, fp,ensure_ascii=False)